# Local Field Potential Analysis
This notebook performs the following steps on LFP data:
1. Filter out 50Hz, 100Hz, and harmonic AC power frequency interference.
2. Use the Hilbert transform to calculate the instantaneous phase for each channel.
3. Perform FFT to find the maximum frequency in the frequency domain for each channel.
4. Simulate the Kuramoto model using the instantaneous phase and maximum frequency.
5. Calculate and plot the order parameter over time.

In [2]:
# Import Required Libraries
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, hilbert
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt

## Load LFP Data
Load the LFP data from a CSV file. Each row represents a channel time series.

In [3]:
# Load LFP data
sampling_rate = 2500  # Hz
lfp_path = "/data1/zhangyuhao/xinchao_data/NP1/20230623_Syt2_conditional_tremor_mice4/LFP/Medial vestibular nucleus_202300622_Syt2_512_2_Day18_P79_g0_t0.exported.imec0.lf.csv"
lfp_data = pd.read_csv(lfp_path, header=None).values
lfp_data = lfp_data.T
num_channels, num_samples = lfp_data.shape
print(f'LFP data loaded with {num_channels} channels and {num_samples} samples.')

LFP data loaded with 78 channels and 6367340 samples.


# bandpass filter

In [7]:
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """带通滤波器"""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    filtered = filtfilt(b, a, data)
    return filtered

# 对每个通道进行带通滤波（例如1.5-50Hz）
filtered_signals = np.array([bandpass_filter(sig, 1.5, 50, 1000) for sig in lfp_data])

## Filter AC Power Frequency Interference
Apply a notch filter to remove 50Hz, 100Hz, and harmonic frequencies.

In [4]:
# Define a notch filter
def notch_filter(data, freq, fs):
    nyquist = 0.5 * fs
    low = (freq - 1) / nyquist
    high = (freq + 1) / nyquist
    b, a = butter(2, [low, high], btype='bandstop')
    return filtfilt(b, a, data)

# Apply the filter for 50Hz, 100Hz, and harmonics
filtered_data = lfp_data.copy()
for freq in [50, 100, 150, 200]:
    for ch in range(num_channels):
        filtered_data[ch, :] = notch_filter(filtered_data[ch, :], freq, sampling_rate)

## Hilbert Transform for Instantaneous Phase
Use the Hilbert transform to calculate the instantaneous phase for each channel.

In [8]:
# Calculate instantaneous phase using Hilbert transform
instantaneous_phase = np.angle(hilbert(filtered_data, axis=1))

## FFT for Maximum Frequency
Perform FFT to find the frequency corresponding to the maximum amplitude in the frequency domain for each channel.

In [9]:
def extract_natural_frequencies(signals, fs):
    """通过FFT提取主导频率"""
    freqs = []
    for sig in signals:
        n = len(sig)
        y = fft(sig)
        xf = fftfreq(n, 1/fs)[:n//2]
        idx = np.argmax(np.abs(y[:n//2]))
        freqs.append(xf[idx])
    return np.array(freqs)

frequencies = extract_natural_frequencies(filtered_data, fs=sampling_rate)

## Simulate Kuramoto Model
Simulate the Kuramoto model using the instantaneous phase and maximum frequency.

In [ ]:
# Simulate Kuramoto model
def kuramoto_model(phases, frequencies, coupling, dt, steps):
    num_oscillators = len(phases)
    phase_history = np.zeros((steps, num_oscillators))
    for t in range(steps):
        phase_history[t, :] = phases
        for i in range(num_oscillators):
            interaction = np.sum(np.sin(phases - phases[i]))
            phases[i] += dt * (frequencies[i] + coupling * interaction / num_oscillators)
    return phase_history

# Initialize parameters
coupling_strength = 0.1
time_step = 0.01
num_steps = 1000
kuramoto_phases = instantaneous_phase[:, 0]

# Run simulation
phase_dynamics = kuramoto_model(kuramoto_phases, frequencies, coupling_strength, time_step, num_steps)

## Calculate and Plot Order Parameter
Calculate the order parameter and plot its dynamics over time.

In [ ]:
# Calculate order parameter
order_parameter = np.abs(np.mean(np.exp(1j * phase_dynamics), axis=1))

# Plot order parameter dynamics
plt.figure(figsize=(10, 6))
plt.plot(np.arange(num_steps) * time_step, order_parameter)
plt.xlabel('Time (s)')
plt.ylabel('Order Parameter')
plt.title('Order Parameter Dynamics')
plt.grid()
plt.show()